# 导入库函数

In [1]:
from __future__ import division
from __future__ import print_function
from gravity_gae.evaluation import compute_scores
from gravity_gae.input_data import load_data
from gravity_gae.model import *
from gravity_gae.optimizer import OptimizerAE, OptimizerVAE
from gravity_gae.preprocessing import *
import numpy as np
import os
import scipy.sparse as sp
import tensorflow as tf
import time

tf.app.flags.DEFINE_string('f', '', 'kernel')#添加的，不报错

# 定义超参数

In [2]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
tf.logging.set_verbosity(tf.logging.ERROR)

flags = tf.app.flags
FLAGS = flags.FLAGS

flags.DEFINE_string('dataset', 'Cornell', 'Name of the graph dataset')

# Select machine learning task to perform on graph
flags.DEFINE_string('task', 'task_3', 'Name of the link prediction task')
''' See section 4.1. of paper for details about tasks:

- task_1: General Directed Link Prediction

- task_2: Biased Negative Samples Directed Link Prediction

- task_3: Bidirectionality Prediction
'''

# Model
flags.DEFINE_string('model', 'source_target_gcn_vae', 'Name of the model')
''' Available Models:

- gcn_ae: Graph Autoencoder from Kipf and Welling (2016), with 2-layer
          GCN encoder and inner product decoder

- gcn_vae: Variational Graph Autoencoder from Kipf and Welling (2016), with
           Gaussian priors, 2-layer GCN encoders and inner product decoder

- source_target_gcn_ae: Source-Target Graph Autoencoder, as introduced
                        in section 2.6 of paper, with 2-layer GCN encoder
                        and asymmetric inner product decoder

- source_target_gcn_vae: Source-Target Graph Variational Autoencoder, as
                         introduced in section 2.6, with Gaussian priors,
                         2-layer GCN encoders and asymmetric inner product
 
- gravity_gcn_ae: Gravity-Inspired Graph Autoencoder, as introduced in
                  section 3.3 of paper, with 2-layer GCN encoder and 
                  gravity-inspired asymmetric decoder
 
- gravity_gcn_vae: Gravity-Inspired Graph Variational Autoencoder, as
                   introduced in section 3.4 of paper, with Gaussian 
                   priors, 2-layer GCN encoders and gravity-inspired decoder
'''

# Model parameters
flags.DEFINE_float('dropout', 0., 'Dropout rate (1 - keep probability).')
flags.DEFINE_integer('epochs', 300, 'Number of epochs in training.')
flags.DEFINE_boolean('features', False, 'Include node features or not in GCN')
flags.DEFINE_float('lamb', 1., 'lambda parameter from Gravity AE/VAE models \
                                as introduced in section 3.5 of paper, to \
                                balance mass and proximity terms')
flags.DEFINE_float('learning_rate', 0.1, 'Initial learning rate (with Adam)')
flags.DEFINE_integer('hidden', 24, 'Number of units in GCN hidden layer.')
flags.DEFINE_integer('dimension', 12, 'Dimension of GCN output: \
- equal to embedding dimension for standard AE/VAE and source-target AE/VAE \
- equal to (embedding dimension - 1) for gravity-inspired AE/VAE, as the \
last dimension captures the "mass" parameter tilde{m}')
flags.DEFINE_boolean('normalize', False, 'Whether to normalize embedding \
                                          vectors of gravity models')
flags.DEFINE_float('epsilon', 0.01, 'Add epsilon to distances computations \
                                       in gravity models, for numerical \
                                       stability')
# Experimental setup parameters
flags.DEFINE_integer('nb_run', 10, 'Number of model run + test')
flags.DEFINE_float('prop_val', 5., 'Proportion of edges in validation set \
                                   (for Task 1)')
flags.DEFINE_float('prop_test', 10., 'Proportion of edges in test set \
                                      (for Tasks 1 and 2)')
flags.DEFINE_boolean('validation', False, 'Whether to report validation \
                                           results  at each epoch (for \
                                           Task 1)')
flags.DEFINE_boolean('verbose', True, 'Whether to print comments details.')

In [3]:
# Lists to collect average results
mean_roc = []
mean_ap = []
mean_time = []


# Load graph dataset
if FLAGS.verbose:
    print("Loading data...")
adj_init, features = load_data(FLAGS.dataset)


# The entire training process is repeated FLAGS.nb_run times
for i in range(FLAGS.nb_run):

    # Edge Masking: compute Train/Validation/Test set
    if FLAGS.verbose:
        print("Masking test edges...")

    if FLAGS.task == 'task_1':
        # Edge masking for General Directed Link Prediction
        adj, val_edges, val_edges_false, test_edges, test_edges_false = \
        mask_test_edges_general_link_prediction(adj_init, FLAGS.prop_test,
                                                FLAGS.prop_val)
    elif FLAGS.task == 'task_2':
        # Edge masking for B.N.S. Directed Link Prediction
        adj, val_edges, val_edges_false, test_edges, test_edges_false = \
        mask_test_edges_biased_negative_samples(adj_init, FLAGS.prop_test)
    elif FLAGS.task == 'task_3':
        # Edge masking for Bidirectionality Prediction
        adj, val_edges, val_edges_false, test_edges, test_edges_false = \
        mask_test_edges_bidirectional_link_prediction(adj_init)
    else:
        raise ValueError('Undefined task!')

    # Preprocessing and initialization
    if FLAGS.verbose:
        print("Preprocessing and Initializing...")
    # Compute number of nodes
    num_nodes = adj.shape[0]
    # If features are not used, replace feature matrix by identity matrix
    if not FLAGS.features:
        features = sp.identity(adj.shape[0])
    # Preprocessing on node features
    features = sparse_to_tuple(features)
    num_features = features[2][1]
    features_nonzero = features[1].shape[0]

    # Define placeholders
    placeholders = {
        'features': tf.sparse_placeholder(tf.float32),
        'adj': tf.sparse_placeholder(tf.float32),
        'adj_orig': tf.sparse_placeholder(tf.float32),
        'dropout': tf.placeholder_with_default(0., shape=())
    }

    # Create model
    model = None
    if FLAGS.model == 'gcn_ae':
        # Standard Graph Autoencoder
        model = GCNModelAE(placeholders, num_features, features_nonzero)
    elif FLAGS.model == 'gcn_vae':
        # Standard Graph Variational Autoencoder
        model = GCNModelVAE(placeholders, num_features, num_nodes,
                            features_nonzero)
    elif FLAGS.model == 'source_target_gcn_ae':
        # Source-Target Graph Autoencoder
        if FLAGS.dimension % 2 != 0:
            raise ValueError('Dimension must be even for Source-Target models')
        model = SourceTargetGCNModelAE(placeholders, num_features,
                                     features_nonzero)
    elif FLAGS.model == 'source_target_gcn_vae':
        # Source-Target Graph Variational Autoencoder
        if FLAGS.dimension % 2 != 0:
            raise ValueError('Dimension must be even for Source-Target models')
        model = SourceTargetGCNModelVAE(placeholders, num_features,
                                      num_nodes, features_nonzero)
    elif FLAGS.model == 'gravity_gcn_ae':
        # Gravity-Inspired Graph Autoencoder
        model = GravityGCNModelAE(placeholders, num_features,
                                  features_nonzero)
    elif FLAGS.model == 'gravity_gcn_vae':
        # Gravity-Inspired Graph Variational Autoencoder
        model = GravityGCNModelVAE(placeholders, num_features, num_nodes,
                                   features_nonzero)
    else:
        raise ValueError('Undefined model!')

    # Optimizer (see tkipf/gae original GAE repository for details)
    pos_weight = float(adj.shape[0] * adj.shape[0] - adj.sum()) / adj.sum()
    norm = adj.shape[0] * adj.shape[0] / float((adj.shape[0] * adj.shape[0]
                                                - adj.sum()) * 2)
    with tf.name_scope('optimizer'):
        # Optimizer for Non-Variational Autoencoders
        if FLAGS.model in ('gcn_ae', 'source_target_gcn_ae', 'gravity_gcn_ae'):
            opt = OptimizerAE(preds = model.reconstructions,
                              labels = tf.reshape(tf.sparse_tensor_to_dense(placeholders['adj_orig'],
                                                                            validate_indices = False), [-1]),
                              pos_weight = pos_weight,
                              norm = norm)
        # Optimizer for Variational Autoencoders
        elif FLAGS.model in ('gcn_vae', 'source_target_gcn_vae', 'gravity_gcn_vae'):
            opt = OptimizerVAE(preds = model.reconstructions,
                               labels = tf.reshape(tf.sparse_tensor_to_dense(placeholders['adj_orig'],
                                                                             validate_indices = False), [-1]),
                               model = model,
                               num_nodes = num_nodes,
                               pos_weight = pos_weight,
                               norm = norm)

    # Normalization and preprocessing on adjacency matrix
    adj_norm = preprocess_graph(adj)
    adj_label = sparse_to_tuple(adj + sp.eye(adj.shape[0]))

    # Initialize TF session
    sess = tf.Session()
    sess.run(tf.global_variables_initializer())

    # Model training
    if FLAGS.verbose:
        print("Training...")
    # Flag to compute total running time
    t_start = time.time()
    for epoch in range(FLAGS.epochs):
        # Flag to compute running time for each epoch
        t = time.time()
        # Construct feed dictionary
        feed_dict = construct_feed_dict(adj_norm, adj_label, features,
                                        placeholders)
        feed_dict.update({placeholders['dropout']: FLAGS.dropout})
        # Weight update
        outs = sess.run([opt.opt_op, opt.cost, opt.accuracy],
                        feed_dict = feed_dict)
        # Compute average loss
        avg_cost = outs[1]
        if FLAGS.verbose:
            # Display epoch information
            print("Epoch:", '%04d' % (epoch + 1), "train_loss=", "{:.5f}".format(avg_cost),
                  "time=", "{:.5f}".format(time.time() - t))
            # Validation (implemented for Task 1 only)
            if FLAGS.validation and FLAGS.task == 'task_1':
                feed_dict.update({placeholders['dropout']: 0})
                emb = sess.run(model.z_mean, feed_dict = feed_dict)
                feed_dict.update({placeholders['dropout']: FLAGS.dropout})
                val_roc, val_ap = compute_scores(val_edges, val_edges_false, emb)
                print("val_roc=", "{:.5f}".format(val_roc), "val_ap=", "{:.5f}".format(val_ap))

    # Compute total running time
    mean_time.append(time.time() - t_start)

    # Get embedding from model
    emb = sess.run(model.z_mean, feed_dict = feed_dict)

    # Test model
    if FLAGS.verbose:
        print("Testing model...")
    # Compute ROC and AP scores on test sets
    roc_score, ap_score = compute_scores(test_edges, test_edges_false, emb)
    # Append to list of scores over all runs
    mean_roc.append(roc_score)
    mean_ap.append(ap_score)

# Report final results
print("\nTest results for", FLAGS.model,
      "model on", FLAGS.dataset, "on", FLAGS.task, "\n",
      "___________________________________________________\n")


print("AP scores \n", mean_ap)
print("Mean AP score: ", np.mean(mean_ap),
      "\nStd of AP scores: ", np.std(mean_ap), "\n \n")

print("AUC scores\n", mean_roc)
print("Mean AUC score: ", np.mean(mean_roc),
      "\nStd of AUC scores: ", np.std(mean_roc), "\n \n")

print("Running times\n", mean_time)
print("Mean running time: ", np.mean(mean_time),
      "\nStd of running time: ", np.std(mean_time), "\n \n")

Loading data...
Masking test edges...
Number of bidirectional edges in the graph: 18.0
Preprocessing and Initializing...
Training...
Epoch: 0001 train_loss= 1.48449 time= 0.10281
Epoch: 0002 train_loss= 1.37191 time= 0.00300
Epoch: 0003 train_loss= 1.34532 time= 0.00301
Epoch: 0004 train_loss= 1.02352 time= 0.00299
Epoch: 0005 train_loss= 1.05719 time= 0.00300
Epoch: 0006 train_loss= 0.94504 time= 0.00304
Epoch: 0007 train_loss= 0.93296 time= 0.00432
Epoch: 0008 train_loss= 0.93606 time= 0.00264
Epoch: 0009 train_loss= 0.93057 time= 0.00308
Epoch: 0010 train_loss= 0.91319 time= 0.00295
Epoch: 0011 train_loss= 0.90837 time= 0.00300
Epoch: 0012 train_loss= 0.86896 time= 0.00307
Epoch: 0013 train_loss= 0.85779 time= 0.00305
Epoch: 0014 train_loss= 0.83132 time= 0.00392
Epoch: 0015 train_loss= 0.83703 time= 0.00317
Epoch: 0016 train_loss= 0.78854 time= 0.00182
Epoch: 0017 train_loss= 0.76885 time= 0.00399
Epoch: 0018 train_loss= 0.75932 time= 0.00308
Epoch: 0019 train_loss= 0.73319 time= 0

Epoch: 0192 train_loss= 0.35887 time= 0.00302
Epoch: 0193 train_loss= 0.36504 time= 0.00196
Epoch: 0194 train_loss= 0.36146 time= 0.00301
Epoch: 0195 train_loss= 0.36607 time= 0.00207
Epoch: 0196 train_loss= 0.36508 time= 0.00346
Epoch: 0197 train_loss= 0.36365 time= 0.00267
Epoch: 0198 train_loss= 0.36336 time= 0.00288
Epoch: 0199 train_loss= 0.36891 time= 0.00304
Epoch: 0200 train_loss= 0.35902 time= 0.00494
Epoch: 0201 train_loss= 0.36926 time= 0.00310
Epoch: 0202 train_loss= 0.36637 time= 0.00290
Epoch: 0203 train_loss= 0.36691 time= 0.00309
Epoch: 0204 train_loss= 0.36405 time= 0.00297
Epoch: 0205 train_loss= 0.36745 time= 0.00299
Epoch: 0206 train_loss= 0.36573 time= 0.00302
Epoch: 0207 train_loss= 0.36387 time= 0.00202
Epoch: 0208 train_loss= 0.37578 time= 0.00292
Epoch: 0209 train_loss= 0.36768 time= 0.00307
Epoch: 0210 train_loss= 0.36676 time= 0.00299
Epoch: 0211 train_loss= 0.35907 time= 0.00195
Epoch: 0212 train_loss= 0.36207 time= 0.00304
Epoch: 0213 train_loss= 0.36296 ti

Epoch: 0092 train_loss= 0.40217 time= 0.00400
Epoch: 0093 train_loss= 0.41028 time= 0.00395
Epoch: 0094 train_loss= 0.39631 time= 0.00299
Epoch: 0095 train_loss= 0.38435 time= 0.00296
Epoch: 0096 train_loss= 0.39660 time= 0.00201
Epoch: 0097 train_loss= 0.40034 time= 0.00398
Epoch: 0098 train_loss= 0.39975 time= 0.00304
Epoch: 0099 train_loss= 0.41050 time= 0.00196
Epoch: 0100 train_loss= 0.40695 time= 0.00401
Epoch: 0101 train_loss= 0.39852 time= 0.00300
Epoch: 0102 train_loss= 0.40381 time= 0.00200
Epoch: 0103 train_loss= 0.39532 time= 0.00304
Epoch: 0104 train_loss= 0.39850 time= 0.00300
Epoch: 0105 train_loss= 0.39942 time= 0.00195
Epoch: 0106 train_loss= 0.40104 time= 0.00301
Epoch: 0107 train_loss= 0.38844 time= 0.00304
Epoch: 0108 train_loss= 0.39801 time= 0.00299
Epoch: 0109 train_loss= 0.39171 time= 0.00300
Epoch: 0110 train_loss= 0.38626 time= 0.00398
Epoch: 0111 train_loss= 0.39402 time= 0.00300
Epoch: 0112 train_loss= 0.41566 time= 0.00200
Epoch: 0113 train_loss= 0.39998 ti

Training...
Epoch: 0001 train_loss= 1.55958 time= 0.08500
Epoch: 0002 train_loss= 1.36990 time= 0.00697
Epoch: 0003 train_loss= 1.28898 time= 0.00296
Epoch: 0004 train_loss= 1.07279 time= 0.00302
Epoch: 0005 train_loss= 0.96966 time= 0.00299
Epoch: 0006 train_loss= 0.97218 time= 0.00204
Epoch: 0007 train_loss= 0.97071 time= 0.00300
Epoch: 0008 train_loss= 0.94048 time= 0.00300
Epoch: 0009 train_loss= 0.96768 time= 0.00302
Epoch: 0010 train_loss= 0.94087 time= 0.00301
Epoch: 0011 train_loss= 0.91504 time= 0.00298
Epoch: 0012 train_loss= 0.88324 time= 0.00305
Epoch: 0013 train_loss= 0.86911 time= 0.00295
Epoch: 0014 train_loss= 0.85331 time= 0.00297
Epoch: 0015 train_loss= 0.84277 time= 0.00301
Epoch: 0016 train_loss= 0.83166 time= 0.00306
Epoch: 0017 train_loss= 0.81414 time= 0.00293
Epoch: 0018 train_loss= 0.79650 time= 0.00305
Epoch: 0019 train_loss= 0.77857 time= 0.00298
Epoch: 0020 train_loss= 0.76228 time= 0.00395
Epoch: 0021 train_loss= 0.74438 time= 0.00200
Epoch: 0022 train_loss

Epoch: 0198 train_loss= 0.35687 time= 0.00405
Epoch: 0199 train_loss= 0.35572 time= 0.00297
Epoch: 0200 train_loss= 0.36980 time= 0.00341
Epoch: 0201 train_loss= 0.35646 time= 0.00201
Epoch: 0202 train_loss= 0.35335 time= 0.00410
Epoch: 0203 train_loss= 0.35274 time= 0.00290
Epoch: 0204 train_loss= 0.35881 time= 0.00299
Epoch: 0205 train_loss= 0.35793 time= 0.00302
Epoch: 0206 train_loss= 0.35534 time= 0.00299
Epoch: 0207 train_loss= 0.35545 time= 0.00310
Epoch: 0208 train_loss= 0.36664 time= 0.00296
Epoch: 0209 train_loss= 0.35822 time= 0.00399
Epoch: 0210 train_loss= 0.35491 time= 0.00395
Epoch: 0211 train_loss= 0.36539 time= 0.00305
Epoch: 0212 train_loss= 0.36348 time= 0.00330
Epoch: 0213 train_loss= 0.36973 time= 0.00281
Epoch: 0214 train_loss= 0.37728 time= 0.00289
Epoch: 0215 train_loss= 0.37688 time= 0.00373
Epoch: 0216 train_loss= 0.37497 time= 0.00305
Epoch: 0217 train_loss= 0.35650 time= 0.00401
Epoch: 0218 train_loss= 0.37283 time= 0.00425
Epoch: 0219 train_loss= 0.37103 ti

...

Epoch: 0075 train_loss= 0.41571 time= 0.00405
Epoch: 0076 train_loss= 0.41580 time= 0.00297
Epoch: 0077 train_loss= 0.40949 time= 0.00297
Epoch: 0078 train_loss= 0.41381 time= 0.00298
Epoch: 0079 train_loss= 0.40919 time= 0.00401
Epoch: 0080 train_loss= 0.41017 time= 0.00299
Epoch: 0081 train_loss= 0.40056 time= 0.00295
Epoch: 0082 train_loss= 0.40980 time= 0.00409
Epoch: 0083 train_loss= 0.40589 time= 0.00296
Epoch: 0084 train_loss= 0.41372 time= 0.00295
Epoch: 0085 train_loss= 0.39589 time= 0.00304
Epoch: 0086 train_loss= 0.38276 time= 0.00302
Epoch: 0087 train_loss= 0.39916 time= 0.00294
Epoch: 0088 train_loss= 0.40177 time= 0.00444
Epoch: 0089 train_loss= 0.38836 time= 0.00202
Epoch: 0090 train_loss= 0.38951 time= 0.00300
Epoch: 0091 train_loss= 0.39767 time= 0.00300
Epoch: 0092 train_loss= 0.40296 time= 0.00196
Epoch: 0093 train_loss= 0.39890 time= 0.00309
Epoch: 0094 train_loss= 0.41735 time= 0.00193
Epoch: 0095 train_loss= 0.38698 time= 0.00201
Epoch: 0096 train_loss= 0.39410 ti

Epoch: 0300 train_loss= 0.34766 time= 0.00318
Testing model...
Masking test edges...
Number of bidirectional edges in the graph: 18.0
Preprocessing and Initializing...
Training...
Epoch: 0001 train_loss= 1.57844 time= 0.13300
Epoch: 0002 train_loss= 1.49677 time= 0.00294
Epoch: 0003 train_loss= 1.20412 time= 0.00307
Epoch: 0004 train_loss= 1.05631 time= 0.00300
Epoch: 0005 train_loss= 0.97219 time= 0.00298
Epoch: 0006 train_loss= 0.99487 time= 0.00301
Epoch: 0007 train_loss= 0.96837 time= 0.00307
Epoch: 0008 train_loss= 0.95815 time= 0.00289
Epoch: 0009 train_loss= 0.96672 time= 0.00403
Epoch: 0010 train_loss= 0.94503 time= 0.00197
Epoch: 0011 train_loss= 0.95244 time= 0.00300
Epoch: 0012 train_loss= 0.90325 time= 0.00401
Epoch: 0013 train_loss= 0.89106 time= 0.00403
Epoch: 0014 train_loss= 0.87371 time= 0.00396
Epoch: 0015 train_loss= 0.83303 time= 0.00305
Epoch: 0016 train_loss= 0.82531 time= 0.00295
Epoch: 0017 train_loss= 0.82480 time= 0.00199
Epoch: 0018 train_loss= 0.80929 time= 

Epoch: 0178 train_loss= 0.36615 time= 0.00397
Epoch: 0179 train_loss= 0.36115 time= 0.00406
Epoch: 0180 train_loss= 0.36789 time= 0.00365
Epoch: 0181 train_loss= 0.37103 time= 0.00305
Epoch: 0182 train_loss= 0.36311 time= 0.00396
Epoch: 0183 train_loss= 0.37237 time= 0.00300
Epoch: 0184 train_loss= 0.36655 time= 0.00301
Epoch: 0185 train_loss= 0.36554 time= 0.00299
Epoch: 0186 train_loss= 0.35914 time= 0.00298
Epoch: 0187 train_loss= 0.35672 time= 0.00402
Epoch: 0188 train_loss= 0.37033 time= 0.00322
Epoch: 0189 train_loss= 0.36679 time= 0.00282
Epoch: 0190 train_loss= 0.36500 time= 0.00204
Epoch: 0191 train_loss= 0.36020 time= 0.00506
Epoch: 0192 train_loss= 0.35654 time= 0.00306
Epoch: 0193 train_loss= 0.36072 time= 0.00285
Epoch: 0194 train_loss= 0.36602 time= 0.00309
Epoch: 0195 train_loss= 0.36187 time= 0.00196
Epoch: 0196 train_loss= 0.35302 time= 0.00301
Epoch: 0197 train_loss= 0.35847 time= 0.00203
Epoch: 0198 train_loss= 0.36069 time= 0.00310
Epoch: 0199 train_loss= 0.37279 ti

Epoch: 0068 train_loss= 0.41058 time= 0.00205
Epoch: 0069 train_loss= 0.42537 time= 0.00304
Epoch: 0070 train_loss= 0.40782 time= 0.00304
Epoch: 0071 train_loss= 0.40560 time= 0.00298
Epoch: 0072 train_loss= 0.40143 time= 0.00302
Epoch: 0073 train_loss= 0.39983 time= 0.00307
Epoch: 0074 train_loss= 0.40804 time= 0.00192
Epoch: 0075 train_loss= 0.41138 time= 0.00306
Epoch: 0076 train_loss= 0.40584 time= 0.00297
Epoch: 0077 train_loss= 0.40697 time= 0.00304
Epoch: 0078 train_loss= 0.40984 time= 0.00204
Epoch: 0079 train_loss= 0.41746 time= 0.00301
Epoch: 0080 train_loss= 0.40524 time= 0.00304
Epoch: 0081 train_loss= 0.40933 time= 0.00299
Epoch: 0082 train_loss= 0.40301 time= 0.00304
Epoch: 0083 train_loss= 0.40552 time= 0.00398
Epoch: 0084 train_loss= 0.40144 time= 0.00300
Epoch: 0085 train_loss= 0.39107 time= 0.00298
Epoch: 0086 train_loss= 0.38890 time= 0.00303
Epoch: 0087 train_loss= 0.39149 time= 0.00297
Epoch: 0088 train_loss= 0.38456 time= 0.00402
Epoch: 0089 train_loss= 0.38416 ti

Epoch: 0289 train_loss= 0.35248 time= 0.00304
Epoch: 0290 train_loss= 0.34641 time= 0.00287
Epoch: 0291 train_loss= 0.34642 time= 0.00305
Epoch: 0292 train_loss= 0.34419 time= 0.00400
Epoch: 0293 train_loss= 0.34836 time= 0.00299
Epoch: 0294 train_loss= 0.34545 time= 0.00300
Epoch: 0295 train_loss= 0.35541 time= 0.00301
Epoch: 0296 train_loss= 0.34945 time= 0.00300
Epoch: 0297 train_loss= 0.35120 time= 0.00302
Epoch: 0298 train_loss= 0.34733 time= 0.00392
Epoch: 0299 train_loss= 0.34781 time= 0.00201
Epoch: 0300 train_loss= 0.34674 time= 0.00404
Testing model...

Test results for source_target_gcn_vae model on Cornell on task_3 
 ___________________________________________________

AP scores 
 [0.5948450603333995, 0.7025397013649212, 0.6300124010450098, 0.7367633027453853, 0.6540385694272566, 0.5916966435219789, 0.6856166099320258, 0.6206463358211606, 0.7416056509709761, 0.6772270100486475]
Mean AP score:  0.6634991285210761 
Std of AP scores:  0.0515618802852753 
 

AUC scores
 [0.515